In [2]:
import sqlite3
import random
import string
import re

In [3]:
def date_of_birth():
    year = random.randint(1960, 1993)
    month = random.randint(1,12)
    day = random.randint(1,30)
    date = f"{year}-{month}-{day}"
    return date

In [4]:
def middle_initial():
    letter = random.choice(string.ascii_letters).upper()
    return letter

In [5]:
def doc_number():
    doc = f"""{random.randint(1,10)}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.randint(1,10)}{random.randint(1,10)}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.randint(1,10)}"""
    return doc

In [6]:
con = sqlite3.connect('../data/universidade_database.db')
cur = con.cursor()

In [6]:
def next_id():
    count = cur.execute("SELECT COUNT(*) FROM person")
    amount = count.fetchall()
    last = amount[0][0]
    next_id = last+1
    return next_id

In [7]:
next_id()

1031

In [8]:
def next_id_teacher():
    count = cur.execute("SELECT COUNT(*) FROM teacher")
    amount = count.fetchall()
    last = amount[0][0]
    next_id = last+1
    return next_id

In [9]:
next_id_teacher()

31

In [11]:
teachers = []

In [12]:
for row in cur.execute("SELECT courseid, teacher FROM course"):
    teachers.append(row[1])

In [13]:
teachers_splited = []
for linha in teachers:
    teachers_splited.append(linha.split())

In [14]:
def first_name(var):
    return teachers_splited[var][0]

In [15]:
def last_name(var):
    return teachers_splited[var][1]

In [161]:
def query(i):
    sqlite_insert_query = f"""INSERT INTO person VALUES ('{next_id()}', '{first_name(i)}', '{middle_initial()}', '{last_name(i)}', '{date_of_birth()}')"""
    return sqlite_insert_query

In [162]:
i=0
for nome in teachers_splited:
    count = cur.execute(query(i))
    con.commit()
    i+=1

In [16]:
cur.execute('SELECT COUNT(*) FROM person')
cur.fetchall()

[(1030,)]

In [17]:
cur.execute('SELECT * FROM person WHERE personid=1001')
cur.fetchall()

[(1001, 'Kellen', 'I', 'Horton', '1965-10-20')]

In [ ]:
cur.execute("DELETE FROM person WHERE personid>1030")

In [39]:
for row in cur.execute(f"SELECT * FROM person"):
        i=0
        for row in cur.execute(f"SELECT * FROM person WHERE firstname='{teachers_splited[i][0]}'"):
            print(row)
            i+=1

(679, 'Kellen', 'L', 'White', '1987-06-25')
(1001, 'Kellen', 'I', 'Horton', '1965-10-20')


In [18]:
i=0
for row in cur.execute(f"SELECT * FROM person WHERE firstname='{teachers_splited[i][0]}'"):
    i+=1
    print(row)

(679, 'Kellen', 'L', 'White', '1987-06-25')
(1001, 'Kellen', 'I', 'Horton', '1965-10-20')


In [19]:
persons = []
for row in cur.execute("SELECT personid, firstname, lastname FROM person"):
    persons.append(row)

In [20]:
len(persons)

1030

In [ ]:
for firstname in persons:
    for name in teachers_splited:
        if firstname[1] == name[0] and firstname[2] == name[1]:
            print(f"{firstname[0]}, {name[0]}, {name[1]}")
            query_teacher = f"""INSERT INTO teacher VALUES ('{next_id_teacher()}', '{firstname[0]}', '{doc_number()}')"""
            count = cur.execute(query_teacher)
            con.commit()

In [8]:
cur.execute("SELECT * FROM teacher")
cur.fetchall()

[(1, 1001, '2QF59ZTF10'),
 (2, 1002, '8IO410ZAJ4'),
 (3, 1003, '6NJ43EWH7'),
 (4, 1004, '10GS104FJY2'),
 (5, 1005, '9RC109MRJ3'),
 (6, 1006, '4IP79NKA3'),
 (7, 1007, '2SP410MBJ4'),
 (8, 1008, '6HL101QWY9'),
 (9, 1009, '1FG110GXR4'),
 (10, 1010, '6AA56IYT3'),
 (11, 1011, '4WJ34HPV3'),
 (12, 1012, '4HY101LMX9'),
 (13, 1013, '5BL102DWP2'),
 (14, 1014, '6HD31YEU5'),
 (15, 1015, '2HR21MWY3'),
 (16, 1016, '10CY51YPD5'),
 (17, 1017, '6DA16NCL2'),
 (18, 1018, '4ZW53PHE5'),
 (19, 1019, '3ZC61OHQ7'),
 (20, 1020, '5SW43IJG9'),
 (21, 1021, '2BS38ONZ4'),
 (22, 1022, '7BZ106HUR2'),
 (23, 1023, '7EK17YHK10'),
 (24, 1024, '1MC11NFP1'),
 (25, 1025, '3RG54SVY1'),
 (26, 1026, '2OM19VCD7'),
 (27, 1027, '8VI25PVB2'),
 (28, 1028, '6JV53JSW5'),
 (29, 1029, '5HW109DDJ4'),
 (30, 1030, '9WK101QEC10')]

In [23]:
cur.execute("SELECT COUNT(*) FROM person")
cur.fetchall()

[(1030,)]

In [24]:
cur.execute("SELECT COUNT(*) FROM teacher")
cur.fetchall()

[(30,)]

In [58]:
con.close()

In [61]:
cur.execute("DELETE FROM teacher WHERE teacherid>30")
con.commit()